In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
from config import gkey

In [2]:
# Import cities file as DataFrame
cities_pd = pd.read_csv("../Resources/cities.csv")
cities_pd.head(100)

,City,State
0,New York City,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Philadelphia,Pennsylvania
...,...,...
95,Fremont,California
96,Baton Rouge,Louisiana
97,Richmond,Virginia
98,Boise,Idaho


In [3]:
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
cities_pd["Lat"] = ""
cities_pd["Lng"] = ""
cities_pd["Airport Name"] = ""
cities_pd["Airport Address"] = ""
cities_pd["Airport Rating"] = ""
cities_pd.head()

print(len(cities_pd))

100


In [4]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in cities_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['City']
    state = row['State']

    # update address key value
    params['address'] = f"{city},{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
cities_pd.head()

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.7128,-74.006,,,
1,Los Angeles,California,34.0522,-118.244,,,
2,Chicago,Illinois,41.8781,-87.6298,,,
3,Houston,Texas,29.7604,-95.3698,,,
4,Philadelphia,Pennsylvania,39.9526,-75.1652,,,


In [5]:
# params dictionary to update each iteration
params = {
    "radius": 48000, 
    "types": "airport",
    "keyword": "international airport",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in cities_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        cities_pd.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [6]:
# Save Data to csv
cities_pd.to_csv("Airport_Output.csv")

# Visualize to confirm airport data appears
cities_pd.head(50)

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.7128,-74.006,John F. Kennedy International Airport,Queens,3.7
1,Los Angeles,California,34.0522,-118.244,Los Angeles International Airport,"1 World Way, Los Angeles",3.5
2,Chicago,Illinois,41.8781,-87.6298,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.6
3,Houston,Texas,29.7604,-95.3698,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",3.8
4,Philadelphia,Pennsylvania,39.9526,-75.1652,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.4
5,Phoenix,Arizona,33.4484,-112.074,Phoenix Sky Harbor International Airport,"3400 E Sky Harbor Blvd, Phoenix",4
6,San Antonio,Texas,29.4241,-98.4936,San Antonio International Airport,"9800 Airport Blvd, San Antonio",4.1
7,San Diego,California,32.7157,-117.161,San Diego International Airport,"3225 N Harbor Dr, San Diego",3.9
8,Dallas,Texas,32.7767,-96.797,Dallas/Fort Worth International Airport,"2400 Aviation Dr, DFW Airport",3.8
9,San Jose,California,37.3382,-121.886,San Francisco International Airport,San Francisco,3.9
